# Technical Factor - Swing Index (SI)

In [1]:
import dai
import pandas as pd

In [2]:
sd = '2025-01-01'
ed = '2026-01-01'

instrument_list = ['600519.SH']

In [3]:
sql = """
WITH
data_base AS (
    SELECT
        date,
        instrument,
        open,
        high,
        low,
        close,
        volume,
    FROM cn_stock_bar1d
),

data_factor AS (
    SELECT
        date,
        instrument,
        GREATEST(high - low, ABS(high - m_lag(close, 1)), ABS(low - m_lag(close, 1))) AS TR,
        50 * (close - m_lag(close, 1) + 0.5 * (close - m_lag(open, 1)) + 0.25 * (m_lag(close, 1) - m_lag(open, 1))) AS SI,
    FROM data_base
),

data_signal_trend AS (
    SELECT
        date,
        instrument,
        IF(SI > 0 AND m_lag(SI, 1) <= 0, 1, 0) AS TRBY1,
        IF(SI < 0 AND m_lag(SI, 1) >= 0, 1, 0) AS TRSL1,
    FROM data_factor
),

data_signal_momentum AS (
    SELECT
        date,
        instrument,
        IF(SI > 0 AND (SI - m_lag(SI, 1)) > 0, 1, 0) AS MTBY1,
        IF(SI < 0 AND (SI - m_lag(SI, 1)) < 0, 1, 0) AS MTSL1,
    FROM data_factor
),

data_signal_reversal AS (
    SELECT
        date,
        instrument,
        IF(SI < -50 AND (SI - m_lag(SI, 1)) > 0, 1, 0) AS RVBY1,
        IF(SI > 50 AND (SI - m_lag(SI, 1)) < 0, 1, 0) AS RVSL1,
    FROM data_factor
),

data_signal_breakout AS (
    SELECT
        date,
        instrument,
        IF(m_lag(SI, 1) <= 100 AND SI > 100, 1, 0) AS BKBY1,
        IF(m_lag(SI, 1) >= -100 AND SI < -100, 1, 0) AS BKSL1,
    FROM data_factor
),

data_combined AS (
    SELECT
        date,
        instrument,
        TR,
        SI,
        TRBY1,
        TRSL1,
        MTBY1,
        MTSL1,
        RVBY1,
        RVSL1,
        BKBY1,
        BKSL1,
    FROM data_factor
    JOIN data_signal_trend USING (date, instrument)
    JOIN data_signal_momentum USING (date, instrument)
    JOIN data_signal_reversal USING (date, instrument)
    JOIN data_signal_breakout USING (date, instrument)
)

SELECT *
FROM data_combined
QUALIFY COLUMNS(*) IS NOT NULL
ORDER BY date, instrument
"""

In [4]:
df = dai.query(sql, filters={"date":[sd,ed], "instrument":instrument_list}).df()
df

,date,instrument,TR,SI,TRBY1,TRSL1,MTBY1,MTSL1,RVBY1,RVSL1,BKBY1,BKSL1
0,2025-01-03,600519.SH,227.922578,-18939.242082,0,0,0,0,0,0,0,0
1,2025-01-06,600519.SH,343.757426,-27339.712360,0,0,0,1,0,0,0,0
2,2025-01-07,600519.SH,105.000787,-3848.942746,0,0,0,0,1,0,0,0
3,2025-01-08,600519.SH,204.951110,42.766031,1,0,1,0,0,0,0,0
4,2025-01-09,600519.SH,257.410774,1680.094056,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
237,2025-12-25,600519.SH,156.945111,7111.311362,1,0,1,0,0,0,1,0
238,2025-12-26,600519.SH,77.205507,2879.368387,0,0,0,0,0,1,0,0
239,2025-12-29,600519.SH,110.909005,-7697.321430,0,1,0,1,0,0,0,1
240,2025-12-30,600519.SH,135.151871,-11622.005073,0,0,0,1,0,0,0,0
